# 7. Transformation des données

## 7.1 Index et colonnes hiérarchiques
La librairie **pandas** permet de gérer des index et des colonnes hiérarchiques. Dans ce cas, les clefs d'accès sont des tuples (ou bien la clef seule pour le premier niveau).

### 7.1.2 Index hiérarchique ou multi-index
Le *DataFrame* des communes comporte implicitement 3 niveaux d'index : *Région*, *Département* et *Code INSEE* (ou le nom de la commune).

In [ ]:
# import des modules usuels
import pandas as pnd
import numpy as np
import matplotlib.pyplot as plt

# commande magique pour l'affichage des graphiques
%matplotlib inline

# options d'affichage
pnd.set_option("display.max_rows", 16)
plt.style.use('seaborn-darkgrid')

In [ ]:
# chargement des données
geo = pnd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                  index_col="Code INSEE")
geo.sort_index(inplace=True)
geo.head()

In [ ]:
geo["Latitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[0]))
geo["Longitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[1]))
geo["Superficie"] /= 100.0
geo["Densité"] = 1000 * geo["Population"] / geo["Superficie"]

# conversion de la colonne Statut en catégorie
# selon version de pandas...

statuts = ["Commune simple", "Chef-lieu canton", "Sous-préfecture",
            "Préfecture", "Préfecture de région", "Capitale d'état"]

if pnd.__version__ < '0.21.0':
    geo["Statut"] = geo["Statut"].astype('category',
                                     categories=statuts, 
                                     ordered=True)
else:
    from pandas.api.types import CategoricalDtype
    cat_statut = CategoricalDtype(categories=statuts, ordered=True)
    geo["Statut"] = geo["Statut"].astype(cat_statut)

In [ ]:
geo.index

On annule l'index existant avec la fonction **reset_index()** :

In [ ]:
# annulation de l'index existant
geo1 = geo.reset_index()
type(geo1.index)

In [ ]:
# on applique un index triple aux communes
geo1.set_index(["Région", "Département", "Code INSEE"], inplace=True)
geo1

In [ ]:
# l'index est devenu un MultiIndex
type(geo1.index)

Accès direct au premier niveau :

In [ ]:
geo1.loc[("ILE-DE-FRANCE")]  # ou bien communes1.loc["ILE-DE-FRANCE"]

Accès au deuxième niveau :

In [ ]:
geo1.loc[("ILE-DE-FRANCE", "HAUTS-DE-SEINE")]

Accès au troisième niveau :

In [ ]:
geo1.loc[("ILE-DE-FRANCE", "HAUTS-DE-SEINE", "92002")]

Tri du multi-index pour éviter les problèmes de performance.

In [ ]:
geo1.sort_index(inplace=True)
geo1.loc[("ILE-DE-FRANCE", "HAUTS-DE-SEINE")]

### Colonnes hiérarchiques

In [ ]:
thd = pnd.read_excel("FranceTHD_Open_Data_Observatoire_Juin2015.xlsx",
                    sheet_name="Communes",
                    header=1,
                    index_col="Code INSEE",
                    names=["Département", "Commune",
                                "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                                "DSL 1 Mbit", "DSL 3 Mbit", "DSL 8 Mbit", "DSL 30 Mbit", "DSL 100 Mbit",
                                "Câble 1 Mbit", "Câble 3 Mbit", "Câble 8 Mbit", "Câble 30 Mbit", "Câble 100 Mbit",
                                "Fibre 1 Mbit", "Fibre 3 Mbit", "Fibre 8 Mbit", "Fibre 30 Mbit", "Fibre 100 Mbit"])
thd.head()

In [ ]:
# on restreint les données au haut débit seul
thd2 = thd.loc[:, "1 Mbit":]
thd2.head()

On constate qu'il y a implicitement 2 niveaux de colonnes : Techno et Débit (ce qu'il y avait dans le fichier Excel initial).

In [ ]:
# on crée des colonnes hiérarchiques que l'on nomme également
thd2.columns = [["THD", "THD", "THD", "THD", "THD",
                     "DSL", "DSL", "DSL", "DSL", "DSL",
                     "Câble", "Câble", "Câble", "Câble", "Câble",
                     "Fibre", "Fibre", "Fibre", "Fibre", "Fibre"],
                   ["1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                   "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                   "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                   "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit"]]
thd2.columns.names = ["Techno", "Débit"]  # on nomme les 2 index"
thd2.head()

On retrouve la structure du fichier Excel initial avec des cellules fusionnées.

In [ ]:
# accès à une valeur
thd2.loc["01002", ("DSL", "3 Mbit")]

In [ ]:
thd2["DSL"].columns

In [ ]:
# accès à une portion de ligne
thd2.loc["01002", ("DSL")]  # ou thd2.loc["01001", "DSL"]

**Remarque** : La liste des colonnes hiérarchiques comporte des répétitions. Il aurait été possible d'utiliser les opérateurs *+* et *\** qui agissent sur les listes :

In [ ]:
[["THD"] * 5 + ["DSL"] * 5 + ["Câble"] * 5 + ["Fibre"] * 5,
 ["1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit"] * 4]

### Permutation de niveaux hiérarchiques

Il est possible d'échanger des niveaux hiérarchiques des index ou des colonnes avec la fonction **swaplevel()** :

In [ ]:
thd2.swaplevel("Débit", "Techno", axis=1).head()

Les niveaux sont bien inversés mais les débits identiques sont éclatés. On verra ci-après comment les rassembler.

## 7.2 Opérations de pivot / Tableaux croisés

La méthode **pivot_table()** construit un tableau synthétique de valeurs agrégées et ventilées selon les différentes valeurs d'une ou plusieurs colonnes.

Elle retourne un nouveau *DataFrame* en fonction des paramètres fournis.

- index : colonnes du *DataFrame* initial dont les valeurs sont utilisées en index
- values : colonnes du *DataFrame* initial dont les valeurs sont agrégées
- columns : colonnes du *DataFrame* initial dont les valeurs sont utilisées en nom de colonnes
- aggfunc : fonction d'agrégation des valeurs, par défaut **numpy.mean** (calcul de la moyenne des valeurs)
- margins : avec ou sans calcul des sous-totaux

Il existe également la fonction **crosstab()** qui calcule par défaut la table des décomptes ou des fréquences croisées.

In [ ]:
# tableau croisé" "Région" x "Statut" avec la somme de "Population"
geo.pivot_table(index="Région",
                columns="Statut",
                values="Population",
                aggfunc='sum')

<div class="alert alert-success">
<b>Exercice 1</b>
<ul>
    <li>Calculez un tableau croisé : régions et altitude moyenne.</li>
    <li>Calculez un tableau croisé : régions x statuts et altitude moyenne.</li>
    <li>Calculez un tableau croisé : régions x statuts et nombre de communes.</li>
</ul>

In [ ]:
# tableau croisé" "Statut" x "Altitude Moyenne", "Population", "Superficie"
geo.pivot_table(index="Statut", values=["Altitude Moyenne", "Population", "Superficie"])

In [ ]:
# fréquences croisées
pnd.set_option('display.precision', 3)
pnd.crosstab(geo["Région"], geo["Statut"], normalize=True, margins=True) * 100

Les méthodes **stack()** et **unstack()** permettent d'empiler les colonnes en index et de dépiler les index en colonnes.

In [ ]:
thd2.head()

La méthode **stack()** prend le niveau de colonne le plus bas et l'envoie au niveau d'index le plus bas.

In [ ]:
# on déplace le niveau de colonne "Débit" vers les index
thd2.stack()

On peut répéter l'opération jusqu'à épuisement des niveaux de colonnes. A la fin on obtient un objet *Series* avec un index hiérarchique.

In [ ]:
# on déplace les niveau de colonne "Débit" et "Techno" vers les index
thd2.stack().stack()

Inversement, la méthode **unstack()** prend le niveau d'index le plus bas et l'envoie au niveau de colonne le plus bas.

In [ ]:
# unstack
thd2.stack().unstack()

Il est possible de préciser le niveau que l'on souhaite empiler ou dépiler : soit en inidiquant le nom de l'index ou de la colonne soit en précisant son rang.

<div class="alert alert-success">
<b>Exercice 2</b>
<ul>
    <li>Trouvez un moyen pour obtenir en colonne les débits rassemblés puis les technos.</li>
</ul>

**Mse à plat des données**

La fonction `melt()` permet de passer d'un format large (nombreuses colonnes) à un format long (nombreuses lignes), en précisant :
- le `DataFrame`
- les colonnes d'index, qui restent inchangées
- les colonnes de valeurs, dont les noms deviendront les valeurs d'une colonne nommée `variable` et les valeurs deviendront les valeurs d'une colonne nommée  `value`.

In [ ]:
# suppresion de l'index
thd.reset_index(inplace=True)
thd.head()

In [ ]:
# utilisation de la fonction melt()
var = pnd.melt(thd,
         id_vars=['Code INSEE', 'Département', 'Commune'],
         value_vars=["1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                                "DSL 1 Mbit", "DSL 3 Mbit", "DSL 8 Mbit", "DSL 30 Mbit", "DSL 100 Mbit",
                                "Câble 1 Mbit", "Câble 3 Mbit", "Câble 8 Mbit", "Câble 30 Mbit", "Câble 100 Mbit",
                                "Fibre 1 Mbit", "Fibre 3 Mbit", "Fibre 8 Mbit", "Fibre 30 Mbit", "Fibre 100 Mbit"])
var

On vérifie la taille du nouveau `DataFrame`.

In [ ]:
# DataFrame initial
thd.shape

In [ ]:
# DataFrame final : nombre de communes x nombre de débits (20)
thd.shape[0] * 20

In [ ]:
# accès à une ligne particulière
var.loc[(var['Commune'] == 'Toulouse') & (var['variable'] == '100 Mbit')]

## 7.3 Fusion d'objets de type *DataFrame*

Dans cette section nous allons fusionner le *DataFrame* des communes avec celui des données sur le haut débit. En utilisant l'index "*Code INSEE*" comme pivot.

La méthode **join()** permet de fusionner 2 objets de type *DataFrame* selon l'index (par défaut) ou selon une colonne.

Il existe une option "*how*" pour spécifier comment s'effectue la jointure (terme utilisé dans les base de données) :
- left (par défaut) : utilise l'index du *DataFrame* dont on invoque la méthode
- right : utilise l'index du *DataFrame* passé en argument
- outer : forme l'union des index
- inner : utilise l'intersection des index

Il existe aussi les options *lsuffix* et *rsuffix* pour suffixer les colonnes dupliquées entre *DataFrame*.

In [ ]:
alldata = geo.join(thd, rsuffix="_")
alldata.head()

In [ ]:
alldata.columns

On constate que *pandas* a suffixé les 2 colonnes "Département" et "Commune" du *DataFrame* de gauche.

Il existe plusieurs autres fonctions de fusion de *DataFrame*.

- fonction **merge()** : jointure sur index ou colonne
- méthode **append()** : ajout d'un objet *Series* ou *DataFrame*
- fonction **concat()** : concaténation de *DataFrames* selon un axe

Voir la documentation : http://pandas.pydata.org/pandas-docs/stable/merging.html

<div class="alert alert-success">
<b>Exercice 3</b>
<ul>
    <li>Construire une table pivot avec les données allant du 1 Mbit au 100 Mbit ventilées par statut des communes.</li>
</ul>

## 7.4 Techniques pour compléter les données manquantes

Le *DataFrame* fait apparaître des lignes incomplètes.

In [ ]:
alldata[alldata.isnull().any(axis=1)]

Nous allons étudier les différentes manières de compléter automatiquement les données manquantes.

In [ ]:
# on remet les statuts en type str
alldata2 = alldata.copy().sort_index()
alldata2["Statut"] = alldata2["Statut"].astype(str)
alldata2

Il est possible de remplacer les *NaN* :
- par une valeur donnée,
- par la dernière valeur connue, option *method='ffill'* (forward fill)
- par la prochaine valeur connue, option *method='bfill'* (backward fill)
- par une fonction appliquée à certaines colonnes
- par une fonction d'interpolation sur l'index

In [ ]:
# on remplace les NaN par une valeur quelconque
alldata2.fillna(0)

In [ ]:
# on remplace les NaN par la dernière valeur connue
alldata2.fillna(method='ffill')

Il est également possible de supprimer les lignes (*axis=0*) ou les colonnes (*axis=1*) comportant un *NaN*.

In [ ]:
# on supprime les lignes avec des valeurs manquantes
alldata2.dropna(axis=0)

Les fonctions de lecture de données comportent également des options permettant de gérer l'absence de valeur.

## 7.5 Techniques pour supprimer les doublons

La méthode **duplicated()** permet de trouver les lignes dupliquées dans un *DataFrame* tandis que la méthode **drop_duplicates()** permet d'en supprimer les lignes dupliquées.

In [ ]:
var = thd.reset_index().loc[:, "1 Mbit":"100 Mbit"]
var

In [ ]:
var.duplicated().value_counts()

In [ ]:
var.drop_duplicates()